## 1. 입력한 질문 벡터화 및 차원축소 

In [1]:
from sentence_transformers import SentenceTransformer

# KoSentence-BERT 모델 불러오기
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [2]:
question = '오른쪽 머리에서 찌릿하는 두통은 뭔가요?'
question_vec = model.encode([question], batchsize=32)

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [3]:
question_vec.shape

(1, 768)

In [4]:
import joblib

# 저장된 RandomForestClassifier 모델 불러오기
rf_model = joblib.load('./headache_randomforest.pkl')
scaler = joblib.load('./headache_scaler.pkl')
pca = joblib.load('./headache_pca.pkl')

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from versio

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

question_ss = scaler.transform(question_vec)
question_pca = pca.transform(question_ss)

In [6]:
# 예측
result = rf_model.predict(question_pca)
result

array([3], dtype=int64)

In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 질문 데이터 불러오기
question_df = pd.read_csv('두통 질문.csv', index_col=0)

le = LabelEncoder()
le_intention = le.fit_transform(question_df['intention'])
le_intention


array([0, 0, 0, ..., 5, 5, 5])

In [8]:
label = le.inverse_transform([result])
label

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array(['증상'], dtype=object)

## 2. faiss
- 대규모 데이터셋에서 빠르고 효율적으로 문장 유사도 검색.
- faiss로 벡터 DB 구축
- 사용자가 입력한 질문을 벡터화하고 faiss를 통해 가장 유사한 벡터 검색

### 질문 임베딩
#### 1) 텍스트 나누기
- 각 언어 모델은 한 번에 처리할 수 있는 텍스트의 길이(토큰수)가 제한적이다.
- 한번에 처리하는 텍스트 길이에 제한을 두어 큰 텍스트를 처리할 때 효율적으로 하기 위함.

In [9]:
import faiss
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
test_sentence = '오른쪽 머리에서 찌릿하는 두통은 뭔가요? 두달에 한 번 정도 머리가 아픈데 도대체 뭐 때문에 아픈지 모르겠습니다. 심각한 문제일까요?'
re.split(r'(?<=[.!?]) +', test_sentence)

['오른쪽 머리에서 찌릿하는 두통은 뭔가요?',
 '두달에 한 번 정도 머리가 아픈데 도대체 뭐 때문에 아픈지 모르겠습니다.',
 '심각한 문제일까요?']

In [11]:
# 2문장씩 문장을 나누는 함수
def split_sentences(sentences, max_sentences=2):
    sentence_list = re.split(r'(?<=[.!?]) +', sentences) # 문장을 마침표, 느낌표, 물음표 뒤에오는 공백을 기준으로 나눔(공백 뒤에 +를 붙여서 하나이상을 의미)
    chunks = []
    for i in range(0, len(sentence_list), max_sentences): # max_sentences 간격만큼
        chunks.append(' '.join(sentence_list[i:i + max_sentences])) # join으로 두개의 문장 공백으로 연결, 리스트 대신 문자열로 생성됨.
    
    return chunks


In [12]:
split_sentences(test_sentence)

['오른쪽 머리에서 찌릿하는 두통은 뭔가요? 두달에 한 번 정도 머리가 아픈데 도대체 뭐 때문에 아픈지 모르겠습니다.',
 '심각한 문제일까요?']

In [13]:
# chunks 속 2문장씩 결합되어있는 문장들의 길이가 너무 긴 경우 나눠주기 위함.
def split_text_with_recursive_splitter(text, chunk_size=300, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size, # 최대 몇자로 설정할지
        chunk_overlap = chunk_overlap # 50개 겹쳐서 시작
    )

    return text_splitter.split_text(text)

In [14]:
split_text_with_recursive_splitter(test_sentence)

['오른쪽 머리에서 찌릿하는 두통은 뭔가요? 두달에 한 번 정도 머리가 아픈데 도대체 뭐 때문에 아픈지 모르겠습니다. 심각한 문제일까요?']

In [15]:
# 질문 불러오기
question_df = pd.read_csv('두통 질문.csv', index_col=0)
question_df

,disease_category,disease_name,intention,question
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?
...,...,...,...,...
3423,응급질환,두통,치료,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...
3424,응급질환,두통,치료,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.
3425,응급질환,두통,치료,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.
3426,응급질환,두통,치료,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?


In [16]:
question_list = question_df['question'].tolist()
intention_list = question_df['intention'].tolist()

In [17]:
test_list = []
a='하이요'
[a]*2

['하이요', '하이요']

In [18]:
test_list.extend([a]*2)
test_list

['하이요', '하이요']

In [19]:
final_chunk = []
question_map = []
intention_map = []

for question in question_list:
    # 2문장씩 나누기
    sentence_chunk = split_sentences(question, max_sentences=2)

    # 긴 문장 쪼개기
    for chunk in sentence_chunk:
        split_chunks = split_text_with_recursive_splitter(chunk, chunk_size=300, chunk_overlap=50)
        final_chunk.extend(split_chunks)

        # 분할된 chunk마다 원본 맵핑
        question_map.extend([question] * len(split_chunks))
        # 분할된 chunk마다 질문 의도 맵핑
        intention = question_df[question_df['question'] == question]['intention'].values[0] # series 형태여서 values로 텍스트만 가져옴.
        intention_map.extend([intention] * len(split_chunks))


In [20]:
question_map[:5]

['두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?',
 '스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?',
 '대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?',
 '갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?',
 '두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?']

In [21]:
intention_map[387]

'원인'

#### 2) 질문 임베딩 및 질문 벡터 DB 만들기

In [22]:
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
embeddings = model.encode(final_chunk)
print('생성된 임베딩 개수: ', len(embeddings)) # 질문은 짧아서 임베딩 개수와 실제 데이터 개수가 같음

생성된 임베딩 개수:  3428


In [23]:
embeddings.shape

(3428, 768)

- 질문 벡터 DB 생성

In [24]:
import numpy as np

# faiss 인덱스 생성 및 데이터 삽입
dimension = embeddings.shape[1] # 임베딩 벡터의 차원
index = faiss.IndexFlatL2(dimension)  # L2 거리(유클리드 거리) 기반으로 검색을 수행하도록 설정(인덱스 생성)
index.add(np.array(embeddings)) # 각 벡터를 L2 거리 기반 인덱스에 추가하고, 검색을 빠르게 할 수 있도록 데이터 구조를 최적화

# 디버깅용 출력: 저장된 벡터 개수 확인
print(f"FAISS 인덱스에 저장된 벡터 개수: {index.ntotal}")

FAISS 인덱스에 저장된 벡터 개수: 3428


## 3. 벡터 DB에서 검색
1) 질문을 받고 의도를 예측한다 (학습시킨 분류 모델 사용)
2) 의도에 해당하는 질문 DB만 추출
3) 임베딩한 질문과 가장 유사한 질문을 추출한 질문 벡터 DB에서 찾는다(faiss 사용)

- 1) 질문 의도 예측 

In [ ]:
question = '오른쪽 머리에서 찌릿하는 두통은 뭔가요?'
question_vec = model.encode([question], batch_size=32)

In [26]:
question_ss = scaler.transform(question_vec)
question_pca = pca.transform(question_ss)
result = rf_model.predict(question_pca)
result

array([3], dtype=int64)

In [27]:
label = le.inverse_transform([result])
label[0]

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


'증상'

- 2) 의도에 해당하는 질문 DB만 추출

In [39]:
# 분류 결과와 같은 증상의 인덱스만 추출
filtered_intention_idx = []
for idx, text in enumerate(intention_map):
    if text == label[0]:
        filtered_intention_idx.append(idx)

len(filtered_intention_idx), filtered_intention_idx

(767,
 [1481,
  1482,
  1483,
  1484,
  1485,
  1486,
  1487,
  1488,
  1489,
  1490,
  1491,
  1492,
  1493,
  1494,
  1495,
  1496,
  1497,
  1498,
  1499,
  1500,
  1501,
  1502,
  1503,
  1504,
  1505,
  1506,
  1507,
  1508,
  1509,
  1510,
  1511,
  1512,
  1513,
  1514,
  1515,
  1516,
  1517,
  1518,
  1519,
  1520,
  1521,
  1522,
  1523,
  1524,
  1525,
  1526,
  1527,
  1528,
  1529,
  1530,
  1531,
  1532,
  1533,
  1534,
  1535,
  1536,
  1537,
  1538,
  1539,
  1540,
  1541,
  1542,
  1543,
  1544,
  1545,
  1546,
  1547,
  1548,
  1549,
  1550,
  1551,
  1552,
  1553,
  1554,
  1555,
  1556,
  1557,
  1558,
  1559,
  1560,
  1561,
  1562,
  1563,
  1564,
  1565,
  1566,
  1567,
  1568,
  1569,
  1570,
  1571,
  1572,
  1573,
  1574,
  1575,
  1576,
  1577,
  1578,
  1579,
  1580,
  1581,
  1582,
  1583,
  1584,
  1585,
  1586,
  1587,
  1588,
  1589,
  1590,
  1591,
  1592,
  1593,
  1594,
  1595,
  1596,
  1597,
  1598,
  1599,
  1600,
  1601,
  1602,
  1603,
  1604,
  

In [29]:
# intention이 증상인지 확인
question_df.iloc[1481]

disease_category                                                 응급질환
disease_name                                                       두통
intention                                                          증상
question            두통의 증상 중 하나로 머리에 맥박이 뛰는 것처럼 일정한 시간차로 통증이 느껴진다면...
Name: 1481, dtype: object

- 3. 임베딩한 질문과 가장 유사한 질문을 추출한 질문 벡터 DB에서 찾음

In [31]:
a = np.array(['a', 'b', 'c', 'd', 'e'])
li = [0, 2]
a[li]

array(['a', 'c'], dtype='<U1')

In [34]:
# 필터링된 인덱스에 해당하는 부분만 벡터 DB로 재생성
filtered_index = faiss.IndexFlatL2(dimension) # 차원 설정
filtered_index.add(np.array(embeddings)[filtered_intention_idx]) # np.array는 리스트를 리스트로 감싸서 필터링 가능

In [43]:
# 생성된 벡터 DB 개수
filtered_index.ntotal

767

In [61]:
fi_distances, fi_indicies = filtered_index.search(question_vec, 5)  # 쿼리 벡터와 가장 유사한(거리가 가까운) 벡터 검색

In [62]:
fi_distances, fi_indicies

(array([[117.60936 , 119.1344  , 123.9859  , 126.939644, 135.4975  ]],
       dtype=float32),
 array([[750, 402,  40, 705, 688]], dtype=int64))

- 새로운 벡터 DB에서 나온 인덱스는 새로운 DB 기준의 인덱스 이므로 원래 벡터 DB에 해당하는 인덱스로 변환해야함.

In [63]:
fi_indicies[0]

array([750, 402,  40, 705, 688], dtype=int64)

In [113]:
filtered_top_idx = [filtered_intention_idx[i] for i in fi_indicies[0]]
filtered_top_idx

[2236, 1886, 1521, 2191, 2173]

In [114]:
for j, i in enumerate(filtered_top_idx):
    # print(j, i)
    print(f'벡터 거리: {fi_distances[0][j]:.2f} \n질문: {question_map[i]}')

벡터 거리: 117.61 
질문: 두통은 머리의 어떤 부위에서 느껴지는 통증을 말하는 건가요?
벡터 거리: 119.13 
질문: 두통과 비슷한 증상이 나타나면 머리가 찌릿한 증상인가요?
벡터 거리: 123.99 
질문: 두통이 발생하면 머리가 찌릿하거나 따끔거리는 것은 정상인가요?
벡터 거리: 126.94 
질문: 두통처럼 머리가 찌릿한 증상은 어떠한 증상으로 나타날 수 있나요?
벡터 거리: 135.50 
질문: 두통으로 인해 머리가 찌릿거리거나 따끔거리는 것이 일반적으로 나타나나요?


## 4. 선별된 질문중 최종 질문 하나 선택
- 질문들의 중앙값을 구하고 해당 중앙값과 가장 유사한 질문 선택

In [115]:
len(embeddings[filtered_top_idx]), embeddings[filtered_top_idx]

(5,
 array([[ 1.3201225 , -1.0341312 , -0.44613767, ..., -0.49889427,
         -0.20969974,  0.8118966 ],
        [ 1.0746241 , -1.7189268 , -0.4708654 , ..., -0.24966142,
         -0.52945894,  1.5034736 ],
        [ 1.1678019 , -1.5875348 , -0.31792077, ..., -0.7242837 ,
         -0.34021705,  0.8577244 ],
        [ 0.9269832 , -1.3945224 , -0.39550015, ..., -0.12050028,
         -0.44697592,  1.337872  ],
        [ 1.0464588 , -1.7999834 , -0.51493835, ..., -0.03640296,
         -0.29288203,  1.149456  ]], dtype=float32))

In [116]:
# 5개 벡터의 중앙값 구하기
filtered_vectors = embeddings[filtered_top_idx]
median_vector = np.median(filtered_vectors, axis=0)
median_vector

array([ 1.07462406e+00, -1.58753479e+00, -4.46137667e-01, -2.20732063e-01,
       -3.41445535e-01,  6.84358597e-01, -6.43548444e-02, -7.32126236e-01,
        7.39271045e-01,  6.40443325e-01, -5.13523281e-01, -3.40488851e-01,
       -7.19415769e-02, -4.99766879e-02,  8.60560834e-01, -1.27377415e+00,
        1.12091148e+00,  1.47653580e+00, -1.31585017e-01,  2.67702788e-01,
       -1.57037365e+00, -8.29324543e-01,  6.44890487e-01,  1.83795333e-01,
        1.04000163e+00, -6.99219644e-01, -4.93974060e-01, -3.43678564e-01,
       -2.28142649e-01,  9.48988616e-01, -8.22070777e-01, -2.21551023e-02,
       -1.08495605e+00,  1.04578674e-01,  1.30219817e-01,  6.33870602e-01,
        1.97457328e-01, -7.98662722e-01, -6.45562336e-02, -1.61316276e+00,
       -2.67953370e-02,  2.88116753e-01, -1.19334543e+00, -4.16712314e-01,
       -8.72129560e-01, -5.51893413e-01, -3.91984701e-01,  7.11529076e-01,
        3.20532560e-01,  2.44787961e-01,  9.02525246e-01, -6.59921914e-02,
        1.21222878e+00, -

- 유사도 비교

In [117]:
final_vec = embeddings[filtered_top_idx]
final_vec.shape

(5, 768)

In [118]:
median_vector.shape

(768,)

In [119]:
# 모양을 맞춰줘야함
reshape_median_vec = median_vector.reshape(1, -1)
reshape_median_vec.shape

(1, 768)

In [120]:
from sklearn.metrics.pairwise import cosine_similarity

# 중앙벡터와 5개의 최종 벡터와의 코사인 유사도 계산
cos_similarities = cosine_similarity(reshape_median_vec, final_vec)
most_similar_idx = np.argmax(cos_similarities) # np.argmax: numpy 배열에서 가장 큰 값의 인덱스 반환
print('유사도가 가장 높은 인덱스: ', most_similar_idx)

유사도가 가장 높은 인덱스:  2


In [122]:
# 실제 벡터 DB에서 찾아야할 인덱스 번호
final_idx = filtered_top_idx[most_similar_idx]
final_idx

1521

In [125]:
print(f'처음 질문: {question}\n최종 선택 질문: {question_map[final_idx]}')

처음 질문: 오른쪽 머리에서 찌릿하는 두통은 뭔가요?
최종 선택 질문: 두통이 발생하면 머리가 찌릿하거나 따끔거리는 것은 정상인가요?


## 5. fine tunning한 LLM 모델에 최종 사용하기 위해 클래스로 만들기

In [18]:
import faiss
import re
import numpy as np
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


class SelectOneQuestion:

    def __init__(self, csvfile, scaler, pca, rf_model, embed_model, labelencoder, question):
        self.csvfile = csvfile
        self.scaler = scaler
        self.pca = pca
        self.rfmodel = rf_model
        self.embed_model = embed_model
        self.le = labelencoder
        
        # 변수 초기화
        self.embeddings = None
        self.dimension = None

    # 벡터 DB 만드는 메서드
    def make_vec_db(self):

        # 2문장씩 문장을 나누는 함수
        def split_sentences(sentences, max_sentences=2):
            sentence_list = re.split(r'(?<=[.!?]) +', sentences) # 문장을 마침표, 느낌표, 물음표 뒤에오는 공백을 기준으로 나눔(공백 뒤에 +를 붙여서 하나이상을 의미)
            chunks = []
            for i in range(0, len(sentence_list), max_sentences): # max_sentences 간격만큼
                chunks.append(' '.join(sentence_list[i:i + max_sentences])) # join으로 두개의 문장 공백으로 연결, 리스트 대신 문자열로 생성됨.
            
            return chunks

        # chunks 속 2문장씩 결합되어있는 문장들의 길이가 너무 긴 경우 나눠주기 위함.
        def split_text_with_recursive_splitter(text, chunk_size=300, chunk_overlap=50):
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size = chunk_size, # 최대 몇자로 설정할지
                chunk_overlap = chunk_overlap # 50개 겹쳐서 시작
            )

            return text_splitter.split_text(text)


        question_df = pd.read_csv(self.csvfile, index_col=0)
        question_list = question_df['question'].tolist()
        intention_list = question_df['intention'].tolist()
        le_intention = self.le.fit_transform(question_df['intention'])


        final_chunk = []
        question_map = []
        intention_map = []

        for question in question_list:
            # 2문장씩 나누기
            sentence_chunk = split_sentences(question, max_sentences=2)

            # 긴 문장 쪼개기
            for chunk in sentence_chunk:
                split_chunks = split_text_with_recursive_splitter(chunk, chunk_size=300, chunk_overlap=50)
                final_chunk.extend(split_chunks)

                # 분할된 chunk마다 원본 맵핑
                question_map.extend([question] * len(split_chunks))
                # 분할된 chunk마다 질문 의도 맵핑
                intention = question_df[question_df['question'] == question]['intention'].values[0] # series 형태여서 values로 텍스트만 가져옴.
                intention_map.extend([intention] * len(split_chunks))

        # 임베딩
        self.embeddings = self.embed_model.encode(final_chunk)
        
        self.dimension = self.embeddings.shape[1] # 임베딩 벡터의 차원
        self.index = faiss.IndexFlatL2(self.dimension)  # L2 거리(유클리드 거리) 기반으로 검색을 수행하도록 설정(인덱스 생성)
        self.index.add(np.array(self.embeddings)) # 각 벡터를 L2 거리 기반 인덱스에 추가하고, 검색을 빠르게 할 수 있도록 데이터 구조를 최적화

        return intention_map, question_map

    def search_one_question(self, question, intention_map, question_map):
        
        #질문 임베딩
        question_vec = self.embed_model.encode([question], batch_size=32)

        question_ss = self.scaler.transform(question_vec)
        question_pca = self.pca.transform(question_ss)
        result = self.rfmodel.predict(question_pca)

        label = self.le.inverse_transform([result])

        # 분류 결과와 같은 증상의 인덱스만 추출
        filtered_intention_idx = []
        for idx, text in enumerate(intention_map):
            if text == label[0]:
                filtered_intention_idx.append(idx)

        # 필터링된 인덱스에 해당하는 부분만 벡터 DB로 재생성
        filtered_index = faiss.IndexFlatL2(self.dimension) # 차원 설정
        filtered_index.add(np.array(self.embeddings)[filtered_intention_idx]) # np.array는 리스트를 리스트로 감싸서 필터링 가능

        fi_distances, fi_indicies = filtered_index.search(question_vec, 5)  # 쿼리 벡터와 가장 유사한(거리가 가까운) 벡터 검색
        filtered_top_idx = [filtered_intention_idx[i] for i in fi_indicies[0]]
        
        # 5개 벡터의 중앙값 구하기
        filtered_vectors = self.embeddings[filtered_top_idx]
        median_vector = np.median(filtered_vectors, axis=0)
        final_vec = self.embeddings[filtered_top_idx]

        # 모양을 맞춰줘야함
        reshape_median_vec = median_vector.reshape(1, -1)

        # 중앙벡터와 5개의 최종 벡터와의 코사인 유사도 계산
        cos_similarities = cosine_similarity(reshape_median_vec, final_vec)
        most_similar_idx = np.argmax(cos_similarities) # np.argmax: numpy 배열에서 가장 큰 값의 인덱스 반환

        final_idx = filtered_top_idx[most_similar_idx]
        
        return question_map[final_idx]

- 생성한 Class를 .py 형식으로 저장

In [27]:
with open('SelectQuestion.py', 'w') as f:
    f.write('''
class SelectOneQuestion:

    def __init__(self, csvfile, scaler, pca, rf_model, embed_model, labelencoder, question):
        self.csvfile = csvfile
        self.scaler = scaler
        self.pca = pca
        self.rfmodel = rf_model
        self.embed_model = embed_model
        self.le = labelencoder
        
        # 변수 초기화
        self.embeddings = None
        self.dimension = None

    # 벡터 DB 만드는 메서드
    def make_vec_db(self):

        # 2문장씩 문장을 나누는 함수
        def split_sentences(sentences, max_sentences=2):
            sentence_list = re.split(r'(?<=[.!?]) +', sentences) # 문장을 마침표, 느낌표, 물음표 뒤에오는 공백을 기준으로 나눔(공백 뒤에 +를 붙여서 하나이상을 의미)
            chunks = []
            for i in range(0, len(sentence_list), max_sentences): # max_sentences 간격만큼
                chunks.append(' '.join(sentence_list[i:i + max_sentences])) # join으로 두개의 문장 공백으로 연결, 리스트 대신 문자열로 생성됨.
            
            return chunks

        # chunks 속 2문장씩 결합되어있는 문장들의 길이가 너무 긴 경우 나눠주기 위함.
        def split_text_with_recursive_splitter(text, chunk_size=300, chunk_overlap=50):
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size = chunk_size, # 최대 몇자로 설정할지
                chunk_overlap = chunk_overlap # 50개 겹쳐서 시작
            )

            return text_splitter.split_text(text)


        question_df = pd.read_csv(self.csvfile, index_col=0)
        question_list = question_df['question'].tolist()
        intention_list = question_df['intention'].tolist()
        le_intention = self.le.fit_transform(question_df['intention'])


        final_chunk = []
        question_map = []
        intention_map = []

        for question in question_list:
            # 2문장씩 나누기
            sentence_chunk = split_sentences(question, max_sentences=2)

            # 긴 문장 쪼개기
            for chunk in sentence_chunk:
                split_chunks = split_text_with_recursive_splitter(chunk, chunk_size=300, chunk_overlap=50)
                final_chunk.extend(split_chunks)

                # 분할된 chunk마다 원본 맵핑
                question_map.extend([question] * len(split_chunks))
                # 분할된 chunk마다 질문 의도 맵핑
                intention = question_df[question_df['question'] == question]['intention'].values[0] # series 형태여서 values로 텍스트만 가져옴.
                intention_map.extend([intention] * len(split_chunks))

        # 임베딩
        self.embeddings = self.embed_model.encode(final_chunk)
        
        self.dimension = self.embeddings.shape[1] # 임베딩 벡터의 차원
        self.index = faiss.IndexFlatL2(self.dimension)  # L2 거리(유클리드 거리) 기반으로 검색을 수행하도록 설정(인덱스 생성)
        self.index.add(np.array(self.embeddings)) # 각 벡터를 L2 거리 기반 인덱스에 추가하고, 검색을 빠르게 할 수 있도록 데이터 구조를 최적화

        return intention_map, question_map

    def search_one_question(self, question, intention_map, question_map):
        
        #질문 임베딩
        question_vec = self.embed_model.encode([question], batch_size=32)

        question_ss = self.scaler.transform(question_vec)
        question_pca = self.pca.transform(question_ss)
        result = self.rfmodel.predict(question_pca)

        label = self.le.inverse_transform([result])

        # 분류 결과와 같은 증상의 인덱스만 추출
        filtered_intention_idx = []
        for idx, text in enumerate(intention_map):
            if text == label[0]:
                filtered_intention_idx.append(idx)

        # 필터링된 인덱스에 해당하는 부분만 벡터 DB로 재생성
        filtered_index = faiss.IndexFlatL2(self.dimension) # 차원 설정
        filtered_index.add(np.array(self.embeddings)[filtered_intention_idx]) # np.array는 리스트를 리스트로 감싸서 필터링 가능

        fi_distances, fi_indicies = filtered_index.search(question_vec, 5)  # 쿼리 벡터와 가장 유사한(거리가 가까운) 벡터 검색
        filtered_top_idx = [filtered_intention_idx[i] for i in fi_indicies[0]]
        
        # 5개 벡터의 중앙값 구하기
        filtered_vectors = self.embeddings[filtered_top_idx]
        median_vector = np.median(filtered_vectors, axis=0)
        final_vec = self.embeddings[filtered_top_idx]

        # 모양을 맞춰줘야함
        reshape_median_vec = median_vector.reshape(1, -1)

        # 중앙벡터와 5개의 최종 벡터와의 코사인 유사도 계산
        cos_similarities = cosine_similarity(reshape_median_vec, final_vec)
        most_similar_idx = np.argmax(cos_similarities) # np.argmax: numpy 배열에서 가장 큰 값의 인덱스 반환

        final_idx = filtered_top_idx[most_similar_idx]
        
        return question_map[final_idx]
    ''')

In [28]:
import faiss
import re
import numpy as np
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
import joblib
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
from SelectQuestion import SelectOneQuestion

# 저장된 RandomForestClassifier 모델 불러오기
rf_model = joblib.load('./headache_randomforest.pkl')
scaler = joblib.load('./headache_scaler.pkl')
pca = joblib.load('./headache_pca.pkl')

csvfile = "두통 질문.csv"
embed_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
labelencoder = LabelEncoder()
question = '오른쪽 머리에서 찌릿하는 두통은 뭔가요?'

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from versio

In [31]:
# 객체 생성
select_question = SelectOneQuestion(csvfile, scaler, pca, rf_model, embed_model, labelencoder, question)

In [32]:
# 벡터 DB 만들기
intention_map, question_map = obj.make_vec_db()

In [33]:
result = obj.search_one_question(question, intention_map, question_map)
result

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


'두통이 발생하면 머리가 찌릿하거나 따끔거리는 것은 정상인가요?'